



Q1 Transformer (10 Points)

Implement an encoder + decoder transformer model to predict next sentence using keras API and train it on shakespeare db, i.e. given one sentence from shakespeare text, it should predict next sentence.

(Feel free to re-use parts of the colab [transformer intro](https://colab.research.google.com/drive/1ZM-JCeCCzgvxJGiqDliXSRvFDBDKMBTB#scrollTo=cbccHV5HWdOz) done in the class.)

Test it on a paragraph (continuous text) with 11 sentences given below:

You feed first sentence. Then repeatedly call the model to predict next sentence, feed the output again to predict next sentence and so on to predict 10 sentences. Compare the last predicted sentence with the actual test paragraph's 11th sentence.


* Make reasonable assumptions about the model architecture and params.
* Build a model from scratch using Keras layers, don't use pretrained models or hugging-face libraries etc.
* Use a small word dictionary and smaller embeddings to reduce the number of params of the model. (Don't use character dictionary.)
* Use special tokens `<SOS>` (start of sentence) and `<EOS>` (end of sentence).
* Use standard positional encoding.


Getting the data

* [dataset](https://www.kaggle.com/datasets/kingburrito666/shakespeare-plays)
* You can just use the [alllines.txt](https://www.kaggle.com/datasets/kingburrito666/shakespeare-plays?select=alllines.txt).


Test paragraph (11 contiguous sentences)

* "Whose end is purposed by the mighty gods?"
* "Yet Caesar shall go forth, for these predictions"
* "Are to the world in general as to Caesar."
* "When beggars die, there are no comets seen,"
* "The heavens themselves blaze forth the death of princes."
* "Cowards die many times before their deaths,"
* "The valiant never taste of death but once."
* "Of all the wonders that I yet have heard."
* "It seems to me most strange that men should fear,"
* "Seeing that death, a necessary end,"
* "Will come when it will come."



In [1]:
from google.colab import files
uploaded = files.upload()

import os
os.rename("alllines.txt", "alllines.txt")
#Here for our assignment we are only focusing on allines..txt


Saving alllines.txt to alllines.txt


Issue and Sucess

- After 13 tries and so much testing for hours and hours there were so much problem with my data and model


Setting up the data: I would say that I added special words, like <start> at the beginning and <end> at the end of each line. This helped the model know where each line starts and stops.

Creating the Model


I added positional which helps the model understand the position of each word in a sentence. This layer gave me some issues at first because it couldnt handle masks


Also, the training data and expected output needed to be exactly the same size, which required careful adjustments to avoid size mismatch errors.
Fine-Tuning and Training

After all these adjustments, I ran the training, keeping an eye out for any issues in the model accuracy and loss how close the correctness.



In [9]:
 #the layer to help the model understand the order of words, as Transformers process all words at once
import tensorflow as tf
import numpy as np

#Masking problem and issue I kept hitting error because some layers didn't support masking so i just made quick
#adjustment. (Please professor please look into this I am so frustrated with the masking)


# I set random seeds
np.random.seed(42)
tf.random.set_seed(42)

#Within this whole assignment stack over flow did not help much but I focus on my tokenization from it

#Remember we got to load the right set


def load_shakespeare_data(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        text = file.read().lower()
    sentences = text.strip().split('\n')


    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

sentences = load_shakespeare_data('alllines.txt')
sentences = ['<start> ' + sentence + ' <end>' for sentence in sentences]
max_vocab_size = 5000
max_seq_len = 50


#Some of the things i use from transformer info were confusing
tokenizer = tf.keras.layers.TextVectorization(
    max_tokens=max_vocab_size,
    output_sequence_length=max_seq_len,
    standardize=None
)
tokenizer.adapt(sentences)
sequences = tokenizer(sentences)
encoder_input = sequences[:-1]
decoder_target = sequences[1:]

train_size = int(len(encoder_input) * 0.8)


encoder_input_train = encoder_input[:train_size]

decoder_target_train = decoder_target[:train_size]

encoder_input_val = encoder_input[train_size:]

decoder_target_val = decoder_target[train_size:]

# I really dont like the issue with padding and sequence
# sequences are correctly aligned was crucial for me


In [8]:
# positional Encoding layer
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, embed_dim, max_seq_len=50):
        super(PositionalEncoding, self).__init__()
        self.embed_dim = embed_dim

        pos = np.arange(max_seq_len)[:, np.newaxis]
        i = np.arange(embed_dim)[np.newaxis, :]
        angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(embed_dim))
        angle_rads = pos * angle_rates

        # using google co lab automation code formated for correct
        angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
        angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

        pos_encoding = angle_rads[np.newaxis, ...]
        self.pos_encoding = tf.cast(pos_encoding, dtype=tf.float32)

    def call(self, x):
        seq_len = tf.shape(x)[1]
        return x + self.pos_encoding[:, :seq_len, :]

# just a single Transformer block
def transformer_block(embed_dim, num_heads, ff_dim, dropout_rate):
    inputs = tf.keras.Input(shape=(None, embed_dim))

    # I encountered issues with attention masks, so I kept them simple.
    attention = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_dim, dropout=dropout_rate
    )(inputs, inputs)

    attention = tf.keras.layers.Dropout(dropout_rate)(attention)
    out1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs + attention)

    # ff network
    ffn = tf.keras.layers.Dense(ff_dim, activation='relu')(out1)


    ffn = tf.keras.layers.Dense(embed_dim)(ffn)


    ffn = tf.keras.layers.Dropout(dropout_rate)(ffn)


    out2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(out1 + ffn)


    return tf.keras.Model(inputs=inputs, outputs=out2)


In [ ]:
#After attention layers, I used feedforward dayers to refine the information further.


#You guys have to rememebr this type of help will allow me to put the attention output so it can pass along better info
embed_dim = 64
num_heads = 2
ff_dim = 256
dropout_rate = 0.1
num_layers = 2

#clean up very quick using google co lab auto
vocab_size = tokenizer.vocabulary_size()
encoder_inputs = tf.keras.Input(shape=(max_seq_len,), name="encoder_inputs")
decoder_inputs = tf.keras.Input(shape=(max_seq_len,), name="decoder_inputs")

#from google auto
embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim, mask_zero=True)


#This step took some troubleshooting because positional encoding can interfere with masking, which raised warnings


# I stacked transformer blocks on the encoder and decoder inputs. At first, I had some shape mismatches in this part

# I compiled the model with a specific loss and optimizer

encoder_embedded = embedding(encoder_inputs)
encoder_embedded = PositionalEncoding(embed_dim, max_seq_len)(encoder_embedded)

decoder_embedded = embedding(decoder_inputs)
decoder_embedded = PositionalEncoding(embed_dim, max_seq_len)(decoder_embedded)

encoder = encoder_embedded
for _ in range(num_layers):
    encoder = transformer_block(embed_dim, num_heads, ff_dim, dropout_rate)(encoder)

decoder = decoder_embedded
for _ in range(num_layers):
    decoder = transformer_block(embed_dim, num_heads, ff_dim, dropout_rate)(decoder)
outputs = tf.keras.layers.Dense(vocab_size, activation='softmax')(decoder)
model = tf.keras.Model([encoder_inputs, decoder_inputs], outputs)

# fix the model with optimizer and loss function
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

decoder_target_train = tf.pad(decoder_target_train[:, 1:], [[0, 0], [0, 1]])
decoder_target_val = tf.pad(decoder_target_val[:, 1:], [[0, 0], [0, 1]])

# I had to ensure that the decoder targets were correctly padded to avoid shape mismatches.

# Training the model for 5 epochs at first due to timming issue. Second I set it 10
EPOCHS = 10
history = model.fit(
    [encoder_input_train, encoder_input_train],
    decoder_target_train,
    validation_data=(
        [encoder_input_val, encoder_input_val],
        decoder_target_val
    ),
    epochs=EPOCHS,
    batch_size=64
)
#I set up an encoder and a decoder, each with a few layers, just like in the Transformer intro. The encoder processes the input sentence,
#while the decoder generates the next sentence based on the encoder’s understanding.


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'positional_encoding_4' (of type PositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'positional_encoding_5' (of type PositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


Epoch 1/10


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/dataset_ops.py:2311: UserWarning: Seed 42 from outer graph might be getting used by function Dataset_map_permutation, if the random op has not been provided any seed. Explicitly set the seed in the function if this is not the intended behavior.
  return map_op._map_v2(


 995/1393 ━━━━━━━━━━━━━━━━━━━━ 5:55 894ms/step - accuracy: 0.7866 - loss: 5.0705

In [ ]:
def generate_text(model, tokenizer, seed_text, max_seq_len):
    # I start with the seed text provided.
    generated = seed_text

    for _ in range(max_seq_len):
        token_list = tokenizer([generated])





        predictions = model.predict([token_list, token_list])

        predicted_id = np.argmax(predictions[0, -1, :])

        predicted_word = tokenizer.get_vocabulary()[predicted_id]

        if predicted_word == '<end>' or predicted_word == '':
            break

        generated += ' ' + predicted_word

    return generated.replace('<start> ', '').replace(' <end>', '')


#8 time! Still issue!

test_paragraph = [
    "Whose end is purposed by the mighty gods?",
    "Yet Caesar shall go forth, for these predictions",
    "Are to the world in general as to Caesar.",
    "When beggars die, there are no comets seen,",
    "The heavens themselves blaze forth the death of princes.",
    "Cowards die many times before their deaths,",
    "The valiant never taste of death but once.",
    "Of all the wonders that I yet have heard.",
    "It seems to me most strange that men should fear,",
    "Seeing that death, a necessary end,"
]
# I ran into errors where my input shapes didn't match the model's expected
#shapes, especially when using the encoder-decoder structure



# My model struggled with accuracy because the training was slow
predicted_sentences = []

for i, sentence in enumerate(test_paragraph):
    generated = generate_text(model, tokenizer, sentence, max_seq_len=20)
    predicted_sentences.append(generated)
    print(f"Sentence {i+1}: {generated}")

print("\nActual 11th sentence:")
print("Will come when it will come.")


This is one of the output for Epoch at 5

I already train the model over 10 times!
# I already train the model over 10 times!


My output just in case you can't run it:


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'positional_encoding' (of type PositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:915: UserWarning: Layer 'positional_encoding_1' (of type PositionalEncoding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(
Epoch 1/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 1316s 940ms/step - accuracy: 0.7909 - loss: 4.3526 - val_accuracy: 0.8484 - val_loss: 0.9906
Epoch 2/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 1389s 974ms/step - accuracy: 0.8466 - loss: 0.9823 - val_accuracy: 0.8488 - val_loss: 0.9342
Epoch 3/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 1346s 966ms/step - accuracy: 0.8472 - loss: 0.9421 - val_accuracy: 0.8491 - val_loss: 0.9225
Epoch 4/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 1299s 933ms/step - accuracy: 0.8475 - loss: 0.9309 - val_accuracy: 0.8493 - val_loss: 0.9171
Epoch 5/5
1393/1393 ━━━━━━━━━━━━━━━━━━━━ 1340s 931ms/step - accuracy: 0.8478 - loss: 0.9245 - val_accuracy: 0.8495 - val_loss: 0.9141
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 545ms/step
Sentence 1: Whose end is purposed by the mighty gods?
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Sentence 2: Yet Caesar shall go forth, for these predictions
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Sentence 3: Are to the world in general as to Caesar.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Sentence 4: When beggars die, there are no comets seen,
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sentence 5: The heavens themselves blaze forth the death of princes.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sentence 6: Cowards die many times before their deaths,
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sentence 7: The valiant never taste of death but once.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Sentence 8: Of all the wonders that I yet have heard.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Sentence 9: It seems to me most strange that men should fear,
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Sentence 10: Seeing that death, a necessary end,

Actual 11th sentence:
Will come when it will come.